In [7]:
import numpy as np 
import pandas as pd
import json 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score

from tqdm import tqdm
tqdm.pandas()
from sklearn.linear_model import ElasticNet
import statsmodels.api as sm
from copy import deepcopy

import Utils
import models

import warnings
warnings.filterwarnings("ignore")

In [11]:
df = pd.read_excel('PMT_march24.xlsx', sheet_name='teens')

In [16]:
df = df.dropna()

In [17]:


df = df.drop(columns=['code'])

seed = 0xAB0BA
np.random.seed(seed)

In [18]:
df

,age,ACC_ADD1,ACC_ADD2,ACC_ADD3,ACC_DIV1,ACC_DIV2,ACC_DIV3,ACC_MUL1,ACC_MUL2,ACC_MUL3,...,N_ADD3,N_DIV1,N_DIV2,N_DIV3,N_MUL1,N_MUL2,N_MUL3,N_SUB1,N_SUB2,N_SUB3
0,15,1.000000,0.894737,0.900000,0.840000,0.444444,0.181818,1.000000,0.666667,0.347826,...,9.0,21.0,4.0,2.0,29.0,6.0,8.0,27.0,9.0,4.0
1,13,0.900000,0.916667,0.714286,0.816327,0.714286,0.625000,0.963636,0.764706,0.500000,...,10.0,40.0,10.0,5.0,53.0,13.0,6.0,50.0,15.0,5.0
2,13,0.878788,0.750000,0.375000,0.782609,0.500000,0.125000,0.916667,0.350000,0.461538,...,3.0,18.0,2.0,1.0,22.0,7.0,6.0,22.0,7.0,4.0
3,10,1.000000,0.500000,0.100000,0.789474,0.263158,0.117647,0.800000,0.222222,0.235294,...,1.0,15.0,5.0,2.0,12.0,2.0,4.0,9.0,5.0,2.0
4,15,0.914286,0.900000,0.714286,0.815789,0.769231,0.600000,0.921053,0.384615,0.545455,...,10.0,31.0,10.0,6.0,35.0,5.0,6.0,39.0,10.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,12,0.944444,1.000000,0.750000,0.956522,0.666667,0.666667,0.950000,0.800000,0.000000,...,3.0,22.0,4.0,2.0,19.0,4.0,0.0,20.0,3.0,5.0
292,17,0.936170,0.826087,0.461538,0.944444,0.750000,0.444444,1.000000,0.722222,0.636364,...,6.0,51.0,9.0,4.0,58.0,13.0,7.0,49.0,15.0,7.0
293,15,0.961538,0.863636,0.631579,0.984375,0.533333,0.800000,0.986111,0.900000,0.307692,...,12.0,63.0,8.0,8.0,71.0,18.0,4.0,63.0,17.0,9.0
294,14,0.954545,0.714286,0.666667,0.900000,0.666667,0.857143,0.937500,0.571429,0.500000,...,6.0,27.0,6.0,6.0,30.0,4.0,2.0,27.0,6.0,3.0


In [19]:
df.columns

Index(['age', 'ACC_ADD1', 'ACC_ADD2', 'ACC_ADD3', 'ACC_DIV1', 'ACC_DIV2',
       'ACC_DIV3', 'ACC_MUL1', 'ACC_MUL2', 'ACC_MUL3', 'ACC_SUB1', 'ACC_SUB2',
       'ACC_SUB3', 'RT_ADD1', 'RT_ADD2', 'RT_ADD3', 'RT_DIV1', 'RT_DIV2',
       'RT_DIV3', 'RT_MUL1', 'RT_MUL2', 'RT_MUL3', 'RT_SUB1', 'RT_SUB2',
       'RT_SUB3', 'N_ADD1', 'N_ADD2', 'N_ADD3', 'N_DIV1', 'N_DIV2', 'N_DIV3',
       'N_MUL1', 'N_MUL2', 'N_MUL3', 'N_SUB1', 'N_SUB2', 'N_SUB3'],
      dtype='object')

In [20]:
def construct_dataset(df, target_column):
    # Determine if the target is aggregated
    
    # Initialize lists to hold columns to include and exclude
    columns_to_exclude = []
    columns_to_include = []
    
    # Base columns categories
    accuracy_columns = [col for col in df.columns if col.startswith('ACC_')]
    reaction_time_columns = [col for col in df.columns if col.startswith('RT_')]
    n_of_corr_columns = [col for col in df.columns if col.startswith('N_') and 'ofCorr' not in col]

    relevant = []
    agg_name = ''
    metric = ''
    if ('ACC' in target_column) or ('Accuracy' in target_column):
        metric = 'ACC'
        relevant = accuracy_columns
        agg_name = 'Accuracy'
    elif 'RT' in target_column:
        metric = 'RT'
        relevant = reaction_time_columns
        agg_name = 'RT'
    elif ('N_' in target_column) or ('ofCorr' in target_column):
        metric = 'N' 
        relevant = n_of_corr_columns
        agg_name = 'NofCorr'


    # For non-aggregated targets, exclude related aggregated columns
    level = target_column.split('_')[-1][-1]
    operation = target_column.split('_')[-1][:-1]
    poss_col = ['ACC', 'RT', 'N']
    possible_X = []
    for l in range(1, 4):
        if l != int(level):
            columns_to_include = []
            for col in poss_col:
                columns_to_include.append(f'{col}_{operation}{l}')
            columns_to_include.append('age')
            X = df[columns_to_include]
            possible_X.append((X, f'from {l}'))

    y = df[target_column]
    
    return possible_X, y


In [21]:
X, y = construct_dataset(df, 'ACC_ADD2')

In [22]:
X[0][0].columns

Index(['ACC_ADD1', 'RT_ADD1', 'N_ADD1', 'age'], dtype='object')

In [23]:
results = {}

for name in tqdm(df.columns):
    if name == 'age':
        continue
    
    Xs, y = construct_dataset(df, name)

    for X, l in Xs:
    
        res = models.LinRegStatmodels(X, y)
        
        r2 = res.rsquared
        params = res.params.index
        
        results[f"{name} {l}"] = res

100%|██████████| 37/37 [00:00<00:00, 58.35it/s]


In [27]:
results.keys()

dict_keys(['ACC_ADD1 from 2', 'ACC_ADD1 from 3', 'ACC_ADD2 from 1', 'ACC_ADD2 from 3', 'ACC_ADD3 from 1', 'ACC_ADD3 from 2', 'ACC_DIV1 from 2', 'ACC_DIV1 from 3', 'ACC_DIV2 from 1', 'ACC_DIV2 from 3', 'ACC_DIV3 from 1', 'ACC_DIV3 from 2', 'ACC_MUL1 from 2', 'ACC_MUL1 from 3', 'ACC_MUL2 from 1', 'ACC_MUL2 from 3', 'ACC_MUL3 from 1', 'ACC_MUL3 from 2', 'ACC_SUB1 from 2', 'ACC_SUB1 from 3', 'ACC_SUB2 from 1', 'ACC_SUB2 from 3', 'ACC_SUB3 from 1', 'ACC_SUB3 from 2', 'RT_ADD1 from 2', 'RT_ADD1 from 3', 'RT_ADD2 from 1', 'RT_ADD2 from 3', 'RT_ADD3 from 1', 'RT_ADD3 from 2', 'RT_DIV1 from 2', 'RT_DIV1 from 3', 'RT_DIV2 from 1', 'RT_DIV2 from 3', 'RT_DIV3 from 1', 'RT_DIV3 from 2', 'RT_MUL1 from 2', 'RT_MUL1 from 3', 'RT_MUL2 from 1', 'RT_MUL2 from 3', 'RT_MUL3 from 1', 'RT_MUL3 from 2', 'RT_SUB1 from 2', 'RT_SUB1 from 3', 'RT_SUB2 from 1', 'RT_SUB2 from 3', 'RT_SUB3 from 1', 'RT_SUB3 from 2', 'N_ADD1 from 2', 'N_ADD1 from 3', 'N_ADD2 from 1', 'N_ADD2 from 3', 'N_ADD3 from 1', 'N_ADD3 from 2',

In [28]:
results['RT_DIV2 from 1'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                RT_DIV2   R-squared (uncentered):                   0.866
Model:                            OLS   Adj. R-squared (uncentered):              0.865
Method:                 Least Squares   F-statistic:                              931.3
Date:                Fri, 22 Mar 2024   Prob (F-statistic):                   1.84e-126
Time:                        13:05:16   Log-Likelihood:                         -748.84
No. Observations:                 290   AIC:                                      1502.
Df Residuals:                     288   BIC:                                      1509.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
ACC_DIV1      13.3714      0.606     22.074      0.000      12.179      14.564
N_DIV1        -0.1134      0.015     -7.692      0.000      -0.142      -0.084
==============================================================================
Omnibus:                       42.267   Durbin-Watson:                   2.171
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               72.789
Skew:                           0.824   Prob(JB):                     1.56e-16
Kurtosis:                       4.818   Cond. No.                         117.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
params_frequency = {}
for name in df.columns:
    params_frequency[name] = set()

params_frequency['const'] = set()
coefs = {}
for res in results:
    for param_name in results[res].params.index:
        params_frequency[param_name].add(res)
    
    coefs[res] = results[res].params.to_dict()

In [30]:

for name in params_frequency:
    new_input = str(params_frequency[name])
    if new_input == 'set()':
        new_input = ''
    params_frequency[name] = new_input

with open("results/params_in_models_prev_level.json", "w") as outfile:
    json.dump(params_frequency, outfile, indent=4)


r2_dict = {}
for name in results:
    r2_dict[name] = results[name].rsquared

with open("results/r2_LinReg_prev_level.json", "w") as outfile:
    json.dump(r2_dict, outfile, indent=4)


rmse_dict = {}
for name in results:
    rmse_dict[name] = float(np.sqrt(results[name].mse_total))

with open("results/rmse_LinReg_prev_level.json", "w") as outfile:
    json.dump(rmse_dict, outfile, indent=4)


with open("results/coefs_LinReg_prev_level.json", "w") as outfile:
    json.dump(coefs, outfile, indent=4)

In [31]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier



In [32]:
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score


In [43]:
result = {}

for name in tqdm(df.columns):
    if name == 'age':
        continue
    
    Xs, y = construct_dataset(df, name)

    for X, l in Xs:
    
        op, rmse, r2, model = models.XGBReg(X, y)
        

        
        result[f"{name} {l}"] = (op, rmse, r2, model)

100%|██████████| 37/37 [06:19<00:00, 10.27s/it]


In [44]:
rmse_dic_xgb = {}
r2_dic_xgb = {}
opt_params = {}
for elem in result:
    r2_dic_xgb[elem] = result[elem][2]
    rmse_dic_xgb[elem] = result[elem][1]
    opt_params[elem] = result[elem][0]
    

In [45]:
r2_dic_xgb

{'ACC_ADD1 from 2': 0.9297993296599387,
 'ACC_ADD1 from 3': 0.8528966249390324,
 'ACC_ADD2 from 1': 0.6643422792277573,
 'ACC_ADD2 from 3': 0.7952388629359304,
 'ACC_ADD3 from 1': 0.3167757419059244,
 'ACC_ADD3 from 2': 0.5119457249957906,
 'ACC_DIV1 from 2': 0.4344013424580653,
 'ACC_DIV1 from 3': 0.46861170693291176,
 'ACC_DIV2 from 1': 0.602157224396406,
 'ACC_DIV2 from 3': 0.28851104314479814,
 'ACC_DIV3 from 1': 0.561597399672056,
 'ACC_DIV3 from 2': 0.7423670449609345,
 'ACC_MUL1 from 2': 0.5107370937368333,
 'ACC_MUL1 from 3': 0.7093020297025885,
 'ACC_MUL2 from 1': 0.7086520140229837,
 'ACC_MUL2 from 3': 0.5573600856966224,
 'ACC_MUL3 from 1': 0.39508658905762795,
 'ACC_MUL3 from 2': 0.6599535873760851,
 'ACC_SUB1 from 2': 0.5500571723000363,
 'ACC_SUB1 from 3': 0.8057490969163894,
 'ACC_SUB2 from 1': 0.6988361545671904,
 'ACC_SUB2 from 3': 0.7837327441167337,
 'ACC_SUB3 from 1': 0.4392611087073409,
 'ACC_SUB3 from 2': 0.6277603374466459,
 'RT_ADD1 from 2': 0.6746828631391097,


In [46]:
with open("results/rmse_XGB_prev_level.json", "w") as outfile:
    json.dump(rmse_dic_xgb, outfile, indent=4)
with open("results/params_XGB_prev_level.json", "w") as outfile:
    json.dump(opt_params, outfile, indent=4)
with open("results/r2_XGB_prev_level.json", "w") as outfile:
    json.dump(r2_dic_xgb, outfile, indent=4)

RF

In [47]:
result = {}

for name in tqdm(df.columns):
    if name == 'age':
        continue
    
    Xs, y = construct_dataset(df, name)

    for X, l in Xs:
    
        op, rmse, r2, model = models.RFReg(X, y)
        

        
        result[f"{name} {l}"] = (op, rmse, r2, model)

  0%|          | 0/37 [00:00<?, ?it/s]

100%|██████████| 37/37 [07:09<00:00, 11.62s/it]


In [48]:
rmse_dic_rf = {}
opt_params = {}
r2_dic_rf = {}

for elem in result:
    r2_dic_rf[elem] = result[elem][2]
    rmse_dic_rf[elem] = result[elem][1]
    opt_params[elem] = result[elem][0]

In [49]:
with open("results/rmse_RF_prev_level.json", "w") as outfile:
    json.dump(rmse_dic_rf, outfile, indent=4)

with open("results/params_RF_prev_level.json", "w") as outfile:
    json.dump(opt_params, outfile, indent=4)

with open("results/r2_RF_prev_level.json", "w") as outfile:
    json.dump(r2_dic_rf, outfile, indent=4)

comparison


In [50]:
with open('results/rmse_RF_prev_level.json') as file:
    res_rf = json.load(file)
with open('results/params_RF_prev_level.json') as file:
    params_rf = json.load(file)

with open('results/rmse_XGB_prev_level.json') as file:
    res_gb = json.load(file)

with open('results/params_XGB_prev_level.json') as file:
    params_gb = json.load(file)

In [61]:
FI_RF = {}
for elem in tqdm(res_rf):
    opt_params = params_rf[elem]
    model = RandomForestRegressor(n_estimators=opt_params[0], max_depth=opt_params[1], min_samples_leaf=opt_params[2], min_samples_split=opt_params[3], n_jobs=-1)

    target_col, l1, l2 = elem.split()
    l = f"{l1} {l2}"
    Xs, y = construct_dataset(df, target_col)
    for X, fr in Xs:
        if fr == l:
            model.fit(X,y)
            FI = list(zip(model.feature_importances_, X.columns))
            FI_RF[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]])


100%|██████████| 72/72 [00:48<00:00,  1.47it/s]


In [62]:

FI_GB = {}
for elem in tqdm(res_rf):
    opt_params = params_gb[elem]
    model = XGBRegressor(n_estimators = opt_params[0], max_depth=opt_params[1], eta=opt_params[2], colsample_bytree=opt_params[3])

    target_col, l1, l2 = elem.split()
    l = f"{l1} {l2}"
    Xs, y = construct_dataset(df, target_col)
    for X, fr in Xs:
        if fr == l:
            model.fit(X,y)

            FI = list(zip(model.feature_importances_, model.get_booster().feature_names))
            FI_GB[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]])

# with open('FeatureImportance.json', 'w') as file:
#     json.dump(FI5, file, indent=4)

100%|██████████| 72/72 [00:11<00:00,  6.28it/s]


In [48]:
with open('results/FI_RF_prev_level.json', 'w') as file:
    json.dump(FI_RF, file, indent=4)

In [63]:
with open('results/FI_GB_prev_level.json', 'w') as file:
    json.dump(FI_GB, file, indent=4)

In [43]:
opt_models = {}
for elem in tqdm(res_rf):
    diff = res_rf[elem] - res_gb[elem]
    if diff > 0:
        opt_params = params_gb[elem]
        model = XGBRegressor(n_estimators = opt_params[0], max_depth=opt_params[1], eta=opt_params[2], colsample_bytree=opt_params[3])
    else:
        opt_params = params_rf[elem]
        model = RandomForestRegressor(n_estimators=opt_params[0], max_depth=opt_params[1], min_samples_leaf=opt_params[2], min_samples_split=opt_params[3], n_jobs=-1)

    X, y = construct_dataset(df, elem)
    model.fit(X,y)

    opt_models[elem] = model  


FI5 = {}
for elem in opt_models:
    if isinstance(opt_models[elem], XGBRegressor):
        FI = list(zip(opt_models[elem].feature_importances_, opt_models[elem].get_booster().feature_names))
        FI5[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]], 'GB')
    else:
        X, y = construct_dataset(df, elem)
        FI = list(zip(opt_models[elem].feature_importances_, X.columns))
        FI5[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]], 'RF')

    
# with open('FeatureImportance.json', 'w') as file:
#     json.dump(FI5, file, indent=4)

  0%|          | 0/58 [00:00<?, ?it/s]

 50%|█████     | 29/58 [00:16<00:16,  1.81it/s]


KeyboardInterrupt: 

In [41]:
with open('FeatureImportance5_new.json', 'w') as file:
     json.dump(FI5, file, indent=4)